In [604]:
#importing required packages
import numpy as np
import pandas as pd 
import seaborn as sns
pd.options.display.float_format='{:.2f}'.format
import math 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick 
from scipy.stats import norm, skew 
from scipy import stats

In [ ]:
#reading the csv file and the first 5 rows of the dataset
telecom_cust = pd.read_csv(r"C:\Users\reddy\Downloads\DIC_dataset.csv") 
telecom_cust.head()

In [ ]:
#The info() tells about the number of columns, column labels, column data types, memory usage, range index, and the number of cells in each column (non-null values).
telecom_cust.info()

In [ ]:
#Dropping customer ID as it only has unique values hence cannot be used in classification
telecom_cust.drop(columns = ['customerID'], inplace = True)
telecom_cust.info()

In [ ]:
#Conversion of the TotalCharges from string to float
l1 = [len(i.split()) for i in telecom_cust['TotalCharges']]
l2 = [i for i in range(len(l1)) if l1[i] != 1]
for i in l2:
    telecom_cust.loc[i,'TotalCharges'] = telecom_cust.loc[(i-1),'TotalCharges']    
telecom_cust['TotalCharges'] = telecom_cust['TotalCharges'].astype(float)
telecom_cust.info()

In [609]:
#label encoding on text data to convcert all the object features into numeric
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = telecom_cust.copy(deep=True)
data_features = [i for i in list(telecom_cust.columns) if i not in list(telecom_cust.describe().columns)]
for i in data_features :
    df[i] = le.fit_transform(df[i])

In [ ]:
df.info()

In [ ]:
#heatmap to check the missing values
sns.heatmap(df.isnull(),cmap = 'magma');

In [ ]:
df.describe() #as the name suggests it does the description and mathematical analysis of each column

In [ ]:
churn=df[df['Churn']==1].describe().T
not_churn=df[df['Churn']==0].describe().T
fig,ax=plt.subplots(nrows=1,ncols=2,figsize=(6,5))
#to plot the mean values of Retained customer features
plt.subplot(1,2,2)
sns.heatmap(not_churn[['mean']],annot=True,linewidths=0.4,linecolor='black',cbar=False,fmt='.2f')
plt.title('Retained Customers');
#to plot the mean values of Non-retained customer features
plt.subplot(1,2,1)
sns.heatmap(churn[['mean']],annot=True,linewidths=0.4,linecolor='black',cbar=False,fmt='.2f')
plt.title('Non-retained Customers');
fig.tight_layout()

In [614]:
#Dividing features into Numerical and Categorical for better visualisation purposes
col=list(df.columns)
categorical_features=[]
numerical_features=[]
for i in col:
    if len(telecom_cust[i].unique())>6:
        numerical_features.append(i)
    else:
        categorical_features.append(i)    

In [ ]:
#plotting Correlation Matrix with all the features using heat map
plt.figure(figsize = (20,5))
sns.heatmap(df.corr(),annot = True);

In [ ]:
#performing correlation of features with target variable instead of all against all like earlier
corr=df.corrwith(df['Churn']).sort_values(ascending=False).to_frame()
corr.columns=['Correlations']
plt.subplots(figsize=(5,5))
sns.heatmap(corr,annot=True,linewidths=0.3,linecolor='black');#plotting the heat map
plt.title('Correlation of features with target variable');#the plotted map title

In [ ]:
#for selecting the best features we are performing on the categorical features(Chi squared test)
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,mutual_info_classif
features=df.loc[:,categorical_features]
# we set the target variable for comparison
target=df.loc[:,'Churn']
best_features=SelectKBest(score_func=chi2,k='all')#selection
fit=best_features.fit(features,target)
featureScores=pd.DataFrame(data=fit.scores_,index=list(features.columns),columns=['Chi Squared Score']) 
plt.subplots(figsize=(5,5))#plots the graph along with the score
sns.heatmap(featureScores.sort_values(ascending=False,by='Chi Squared Score'),annot=True,linewidths=0.4,linecolor='black',fmt='.2f');

In [ ]:
#for selecting the best features we are performing on the numerical features(Anova)
from sklearn.feature_selection import f_classif
features=df.loc[:,numerical_features]
target=df.loc[:,'Churn']#setting the target variable for comparision
best_features=SelectKBest(score_func=f_classif,k='all')#selection
fit=best_features.fit(features,target)
featureScores=pd.DataFrame(data=fit.scores_,index=list(features.columns),columns=['ANOVA Score']) 
plt.subplots(figsize=(5,5))#plots the graph along with the score
sns.heatmap(featureScores.sort_values(ascending=False,by='ANOVA Score'),annot = True,linewidths = 0.4,linecolor = 'black',fmt = '.2f');

In [ ]:
#dropping the non important variables
df.drop(columns = ['PhoneService', 'gender','StreamingTV','StreamingMovies','MultipleLines','InternetService'],inplace = True)
df.info()

In [ ]:
#Target variable - Churn Visualization by plotting a pie chart 
l=list(df['Churn'].value_counts())# list to hold all the tuple values of the churn column
circle=[l[0]/sum(l)*100,l[1]/sum(l)*100]
fig=plt.subplots(nrows=1,ncols=1,figsize=(20,5))# placement of the graph coordinates in the output
plt.subplot(1,2,1)
plt.pie(circle,labels=['Retained Customer','Non-retained Customer'],autopct='%1.1f%%',startangle=90,
       wedgeprops = {'linewidth': 1,'edgecolor' : 'black','antialiased' : True})# commands to obtain the desired comparison of the target variable 
plt.title('Retained - Non-retained %');#title of the chart

In [ ]:
#Plotting bar graph to view the features after removing the unnecessary variables.
plt.figure(figsize=(15,5))
df.corr()['Churn'].sort_values(ascending=False).plot(kind='bar')

In [622]:
#removing churn for visualisation purposes
categorical_features.remove('Churn')

In [623]:
# breaking the features into groups
# Customer Information
l1=['SeniorCitizen','Partner','Dependents']
# Services Signed Up for!
l2=['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport'] 
# Payment Information
l3=['Contract','PaperlessBilling','PaymentMethod'] 

In [ ]:
# plotting histograms of Customer Information vs target variable
fig=plt.subplots(nrows=1,ncols=1,figsize=(15,10))# placement of the graph coordinates in the output
for i in range(len(l1)):
    plt.subplot(2,2,i+1) #runs on loop for each each feature
    ax=sns.countplot(l1[i],data=df,hue="Churn",edgecolor='black')#specifies each histogram and their description from the dataset
    for rect in ax.patches:
        ax.text(rect.get_x()+rect.get_width()/2,rect.get_height()+2,rect.get_height(),horizontalalignment='center',fontsize=11)
    title=l1[i]+'vs Churn'
    plt.title(title);

In [ ]:
#Histogram of Services Subscribed by the Customer vs the Target variable
fig=plt.subplots(nrows=2,ncols=2,figsize=(20,14))#placement of the graph coordinates in the output
for i in range(len(l2[-4:])):
    plt.subplot(2,2,i+1)#runs on loop for each each feature
    ax=sns.countplot(l2[-4+i],data=df,hue="Churn",edgecolor='black')#specifies each histogram and their description from the dataset
    for rect in ax.patches:
        ax.text(rect.get_x()+rect.get_width()/2,rect.get_height()+2,rect.get_height(),horizontalalignment='center',fontsize=11)
    title=l2[-4+i]+ 'vs Churn'#listing the name of each feature
    plt.title(title);

In [ ]:
#Histogram of Payment Information of the Customer vs the Target variable
fig=plt.subplots(nrows=1,ncols=3,figsize=(25,7))#placement of the graph coordinates in the output
plt.subplot(1,3,1)#contract feature
ax=sns.countplot(l3[0],data=df,hue="Churn",edgecolor='black')
for rect in ax.patches:
    ax.text(rect.get_x()+rect.get_width()/2,rect.get_height()+2,rect.get_height(),horizontalalignment='center',fontsize=11)
title=l3[0]+' vs Churn'
plt.title(title);
plt.subplot(1,3,2)#paperless billing feature
ax=sns.countplot(l3[1],data=df,hue="Churn",edgecolor='black')
for rect in ax.patches:
    ax.text(rect.get_x()+rect.get_width()/2,rect.get_height()+2,rect.get_height(),horizontalalignment='center',fontsize=11)
title=l3[1]+' vs Churn'
plt.title(title);
plt.subplot(1,3,3)#payment method feature
ax=sns.countplot(l3[2],data=df,hue="Churn",edgecolor='black')
for rect in ax.patches:
    ax.text(rect.get_x()+rect.get_width()/2,rect.get_height()+2,rect.get_height(),horizontalalignment='center',fontsize=11)
title=l3[2]+' vs Churn'
plt.title(title);

In [ ]:
#Analyzing the Numerical Variables(Tenure, Monthly Charges, Total Charges) with respect to churn
plot,ax=plt.subplots(2,3,figsize=(20,10))
#plotting histogram and distributions of each numerical feature wrt churn
g=sns.histplot(df['tenure'],kde=True,ax=ax[0][0])
res=stats.probplot(df['tenure'],plot=ax[1][0])
g=sns.histplot(df['MonthlyCharges'],kde=True,ax=ax[0][1])
res=stats.probplot(df['MonthlyCharges'],plot=ax[1][1])
g=sns.histplot(df['TotalCharges'],kde=True,ax=ax[0][2])
res=stats.probplot(df['TotalCharges'],plot=ax[1][2])

In [ ]:
#Performing normalisation of each numerical feature
from sklearn.preprocessing import MinMaxScaler,StandardScaler
mms = MinMaxScaler() # Normalization
df['tenure']=mms.fit_transform(df[['tenure']])
df['MonthlyCharges']=mms.fit_transform(df[['MonthlyCharges']])
df['TotalCharges']=mms.fit_transform(df[['TotalCharges']])
# outlier detection using box plots
plt.figure(figsize=(16,4))
sns.boxplot(data=df[['tenure','MonthlyCharges','TotalCharges']])

In [ ]:
#Visualizing against Churn
def plot(feature):#defining a method to plot the distribution graph of the numerical features against churn
    plt.figure(figsize=(10, 3))
    plt.title("Distribution for {}".format(feature))
    ax0=sns.kdeplot(df[df['Churn']==0][feature].dropna(),label='Churn: No')
    ax1=sns.kdeplot(df[df['Churn']==1][feature].dropna(),label='Churn: Yes')
    ax0.legend(["Not Churn","Churn"],loc='upper right')
plot('tenure')
plot('MonthlyCharges')
plot('TotalCharges')

In [ ]:
#Visualization of Numerical Features w.r.t Target Variable (Churn)
fig,ax=plt.subplots(nrows=3,ncols=1,figsize=(15,15))#the coordinates at which the plots need to be placed in the output
for i in range(len(numerical_features)):# loop to run each of teh three numerical features
    plt.subplot(3,1,i+1)
    sns.countplot(numerical_features[i],data=df,hue="Churn",edgecolor='black')
    plt.legend(['No Churn','Churn'],loc = 'upper left')
    title=numerical_features[i]+' w.r.t Churn'
    plt.title(title);

In [633]:
#to balance the data using SMOTE
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over=SMOTE(sampling_strategy=1)
f1=df.iloc[:,:13].values
t1=df.iloc[:,13].values
f1,t1=over.fit_resample(f1,t1)
Counter(t1)

Counter({0: 5174, 1: 5174})

In [ ]:
#Modelling phase starts here

In [ ]:
#PHASE 2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_curve

In [ ]:
#after trying differenet split ratios, we decided on test size of 20% of the dataset.
x_train, x_test, y_train, y_test = train_test_split(f1, t1, test_size = 0.20, random_state = 2)

In [ ]:
#applying logistic regression on the data set
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector
selector = SequentialFeatureSelector(estimator=LogisticRegression(random_state=0),
                                    n_features_to_select=6,
                                    direction='forward',
                                    scoring='roc_auc',
                                    cv=5)
selector.fit(x_train,y_train)
x_train = selector.transform(x_train)
x_test = selector.transform(x_test)
log_reg = LogisticRegression(random_state=0)
log_reg.fit(x_train, y_train)
y_pred = log_reg.predict(x_test)
#to print the confusion matrix
cm = confusion_matrix(y_test,y_pred)
names = ['True Negative','False Positive','False Negetive','True Positive']
counts = [value for value in cm.flatten()]
percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm,annot = labels,cmap = 'Reds',fmt ='')
#to print the report to evaluate the model 
print(classification_report(y_test,y_pred))
print('\n')
print(accuracy_score(y_test,y_pred))

In [ ]:
#decision tree classifier
from sklearn.tree import DecisionTreeClassifier
classifier_dt = DecisionTreeClassifier(random_state = 1000,max_depth = 4,min_samples_leaf = 1)

#to print the ROC AUC and the cross-validation scores along with the plot
classifier_dt.fit(x_train,y_train)
prediction=classifier_dt.predict(x_test)
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
print('\n')
print("Cross Validation Score : ",'{0:.2%}'.format(cross_val_score(classifier_dt,x_train,y_train,cv=cv,scoring='roc_auc').mean()))
print("ROC_AUC Score : ",'{0:.2%}'.format(roc_auc_score(y_test,prediction)))
print('\n')
plot_roc_curve(classifier_dt,x_test,y_test)
plt.title('ROC_AUC_Plot')
plt.show()

# Confusion Matrix
cm=confusion_matrix(y_test,classifier_dt.predict(x_test))
names=['True Negatve','False Positive','False Negative','True Positive']
counts=[value for value in cm.flatten()]
percentages=['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
labels=[f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
labels=np.asarray(labels).reshape(2,2)
sns.heatmap(cm,annot=labels,cmap='Reds',fmt='')
    
# Classification Report
print(classification_report(y_test,classifier_dt.predict(x_test)))
print('\n')
print(accuracy_score(y_test,classifier_dt.predict(x_test)))

In [ ]:
#random  forest classifier
!pip install sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier
classifier_rf=RandomForestClassifier(max_depth=4,random_state=0)
#prints the cross validation score, ROC_AUC Score and the ROC_AUC plot

#to print the ROC AUC and the cross-validation scores along with the plot
classifier_rf.fit(x_train,y_train)
prediction=classifier_rf.predict(x_test)
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
print('\n')
print("Cross Validation Score : ",'{0:.2%}'.format(cross_val_score(classifier_rf,x_train,y_train,cv=cv,scoring='roc_auc').mean()))
print("ROC_AUC Score : ",'{0:.2%}'.format(roc_auc_score(y_test,prediction)))
print('\n')
plot_roc_curve(classifier_rf,x_test,y_test)
plt.title('ROC_AUC_Plot')
plt.show()

# Confusion Matrix
cm=confusion_matrix(y_test,classifier_rf.predict(x_test))
names=['True Negatve','False Positive','False Negative','True Positive']
counts=[value for value in cm.flatten()]
percentages=['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
labels=[f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
labels=np.asarray(labels).reshape(2,2)
sns.heatmap(cm,annot=labels,cmap='Reds',fmt='')
    
# Classification Report
print(classification_report(y_test,classifier_rf.predict(x_test)))
print('\n')
print(accuracy_score(y_test,classifier_rf.predict(x_test)))

In [ ]:
#xgboost classifier
! pip install xgboost
from xgboost import XGBClassifier
classifier_xgb=XGBClassifier(learning_rate=0.01,max_depth=3,n_estimators=1000)

#to print the ROC AUC and the cross-validation scores along with the plot
classifier_xgb.fit(x_train,y_train)
prediction=classifier_xgb.predict(x_test)
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
print('\n')
print("Cross Validation Score : ",'{0:.2%}'.format(cross_val_score(classifier_xgb,x_train,y_train,cv=cv,scoring='roc_auc').mean()))
print("ROC_AUC Score : ",'{0:.2%}'.format(roc_auc_score(y_test,prediction)))
print('\n')
plot_roc_curve(classifier_xgb,x_test,y_test)
plt.title('ROC_AUC_Plot')
plt.show()

# Confusion Matrix
cm=confusion_matrix(y_test,classifier_xgb.predict(x_test))
names=['True Negatve','False Positive','False Negative','True Positive']
counts=[value for value in cm.flatten()]
percentages=['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
labels=[f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
labels=np.asarray(labels).reshape(2,2)
sns.heatmap(cm,annot=labels,cmap='Reds',fmt='')
    
# Classification Report
print(classification_report(y_test,classifier_xgb.predict(x_test)))
print('\n')
print(accuracy_score(y_test,classifier_xgb.predict(x_test)))

In [ ]:
#lightgbm classifier
!pip install lightgbm
from lightgbm import LGBMClassifier
classifier_lgbm=LGBMClassifier(learning_rate=0.01,max_depth=3,n_estimators=1000)
#to print the ROC AUC and the cross-validation scores along with the plot
classifier_lgbm.fit(x_train,y_train)
prediction=classifier_lgbm.predict(x_test)
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
print('\n')
print("Cross Validation Score : ",'{0:.2%}'.format(cross_val_score(classifier_lgbm,x_train,y_train,cv=cv,scoring='roc_auc').mean()))
print("ROC_AUC Score : ",'{0:.2%}'.format(roc_auc_score(y_test,prediction)))
print('\n')
plot_roc_curve(classifier_lgbm,x_test,y_test)
plt.title('ROC_AUC_Plot')
plt.show()

# Confusion Matrix
cm=confusion_matrix(y_test,classifier_lgbm.predict(x_test))
names=['True Negatve','False Positive','False Negative','True Positive']
counts=[value for value in cm.flatten()]
percentages=['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
labels=[f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
labels=np.asarray(labels).reshape(2,2)
sns.heatmap(cm,annot=labels,cmap='Reds',fmt='')
    
# Classification Report
print(classification_report(y_test,classifier_lgbm.predict(x_test)))
print('\n')
print(accuracy_score(y_test,classifier_lgbm.predict(x_test)))